In [10]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo_request al host pasado por parámetro. La función devuelve un diccionario que contiene la siguiente información:

* dst: Host contactado
* response: True si se obtuvo respuesta; False en caso contrario
* rtt_us: RTT en milisegundos (None si no se obtuvo respuesta)
* len: Longitud del paquete recibido (None si no se obtuvo respuesta)
* ttl: Time to live del paquete recibido (None si no se obtuvo respuesta)


In [11]:
def ping(host): 

    # Diccionario de códigos ICMP para respuestas de tipo 3 (Destino inalcanzable)
    codigos = {
        0: "Red de destino inalcanzable",
        1: "Host de destino inalcanzable",
        2: "Protocolo inalcanzable",
        3: "Puerto inalcanzable",
        4: "Fragmentación requerida y DF activado",
        5: "Ruta de origen fallida"}
    
    # Creo diccionario con valores por defecto
    data = {
        "dst": host,
        "response_type": None,
        "rtt_ms": None,
        "len": None,
        "ttl": None
    }
    
    print(f"Pinging {host}...")

    packet = IP(dst=host)/ICMP(type=8) # Echo Request
    
    inicio = time.time()
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    fin = time.time()
    
    if response:

        icmp_layer = response.getlayer(ICMP)

        code = icmp_layer.code

        rtt_ms = (fin - inicio) * 1000 # RTT en milisegundos
        data["rtt_ms"] = rtt_ms
        data["len"] = response.len
        data["ttl"] = response.ttl
        data["response_type"] = response.type

        if response.type == 3: 
            print(f"Destino {host} inalcanzable: " + codigos[code])
        
        elif response.type == 0: # Si la respuesta es echo reply
            print(f"Echo-reply recibida de {host}")

        else:
            print(f"Respuesta ICMP recibida de tipo {response.type}, código {icmp_layer.code}")

    else:
        print(f"No se recibió respuesta de {host}")

    return data

In [12]:
def ping_multiple(host, cantidad_paquetes):

    res = dict()
    sent_packets = cantidad_paquetes
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    lista_rtts = []
    
    for i in range(cantidad_paquetes):

        resultado = ping(host)

        # print(resultado) # podemos descomentar para ver el resultado de cada ping

        if resultado["response_type"] == 0: 
            received_packets += 1
            total_rtt += resultado["rtt_ms"]
            min_rtt = min(min_rtt, resultado["rtt_ms"])
            max_rtt = max(max_rtt, resultado["rtt_ms"])
            lista_rtts.append(resultado["rtt_ms"])

        else:
            lost_packets += 1 # Si el destino es inalcanzable o no se recibe respuesta, el ping se considera perdido

    res["host"] = host
    res["Paquetes enviados"] = sent_packets
    res["Paquetes recibidos"] = received_packets
    res["Paquetes perdidos"] = lost_packets
    res["Porcentaje de paquetes perdidos"] = (lost_packets / sent_packets) 
    res["RTT total"] = total_rtt if received_packets > 0 else None
    res["RTT mínimo"] = min_rtt if min_rtt != float('inf') else None
    res["RTT máximo"] = max_rtt if max_rtt != float('-inf') else None
    res["RTT promedio"] = total_rtt / received_packets if received_packets > 0 else None
    res["Desvió estándar del RTT promedio"] = statistics.stdev(lista_rtts) if len(lista_rtts) > 1 else None
    
    return res

Contactamos a 5 universidades de continentes distintos. También contactamos a UTDT para ver las diferencias.

In [13]:
ping_multiple('utdt.edu.ar', 5)


Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar
Pinging utdt.edu.ar...
Echo-reply recibida de utdt.edu.ar


{'host': 'utdt.edu.ar',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 492.8760528564453,
 'RTT mínimo': 65.11902809143066,
 'RTT máximo': 125.74291229248047,
 'RTT promedio': 98.57521057128906,
 'Desvió estándar del RTT promedio': 30.527201587399055}

In [14]:
ping_multiple('mit.edu', 5)

Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu
Pinging mit.edu...
Echo-reply recibida de mit.edu


{'host': 'mit.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 520.2734470367432,
 'RTT mínimo': 58.95209312438965,
 'RTT máximo': 152.32300758361816,
 'RTT promedio': 104.05468940734863,
 'Desvió estándar del RTT promedio': 41.237935097721476}

In [15]:
ping_multiple('ie.edu', 5)

Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu
Pinging ie.edu...
Echo-reply recibida de ie.edu


{'host': 'ie.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 262.9408836364746,
 'RTT mínimo': 19.60587501525879,
 'RTT máximo': 120.11170387268066,
 'RTT promedio': 52.58817672729492,
 'Desvió estándar del RTT promedio': 40.91292658513064}

In [16]:
ping_multiple('sydney.edu.au', 5)

Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au
Pinging sydney.edu.au...
Echo-reply recibida de sydney.edu.au


{'host': 'sydney.edu.au',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 2048.149585723877,
 'RTT mínimo': 342.5297737121582,
 'RTT máximo': 549.3576526641846,
 'RTT promedio': 409.6299171447754,
 'Desvió estándar del RTT promedio': 92.29536654628816}

In [17]:
ping_multiple('cuhk.edu.cn', 5)

Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn
Pinging cuhk.edu.cn...
Echo-reply recibida de cuhk.edu.cn


{'host': 'cuhk.edu.cn',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 1716.3801193237305,
 'RTT mínimo': 331.2041759490967,
 'RTT máximo': 357.44190216064453,
 'RTT promedio': 343.2760238647461,
 'Desvió estándar del RTT promedio': 13.07432881450729}

In [18]:
ping_multiple('uniben.edu', 5)

Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu
Pinging uniben.edu...
Echo-reply recibida de uniben.edu


{'host': 'uniben.edu',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 5,
 'Paquetes perdidos': 0,
 'Porcentaje de paquetes perdidos': 0.0,
 'RTT total': 396.09313011169434,
 'RTT mínimo': 21.183013916015625,
 'RTT máximo': 186.10000610351562,
 'RTT promedio': 79.21862602233887,
 'Desvió estándar del RTT promedio': 63.15631002447692}

In [19]:
ping_multiple('alexu.edu.eg', 5)

Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg
Pinging alexu.edu.eg...
No se recibió respuesta de alexu.edu.eg


{'host': 'alexu.edu.eg',
 'Paquetes enviados': 5,
 'Paquetes recibidos': 0,
 'Paquetes perdidos': 5,
 'Porcentaje de paquetes perdidos': 1.0,
 'RTT total': None,
 'RTT mínimo': None,
 'RTT máximo': None,
 'RTT promedio': None,
 'Desvió estándar del RTT promedio': None}